# Version VII - Ensemble Test (10 models, Dataset Only)

### Imports and initial setup

In [1]:
import import_ipynb

from InitialSetup import dataset, dataset_length, BATCH_SIZE, tf, prepare_callbacks, test_dataset_length, AUTOTUNE, train_models, load_weights, load_weights_read, evaluate_models, get_labels_logits_and_preds, get_class_preds, get_class_from_sum_of_logits, np, BatchNormalization, LeakyReLU

NUM_MODELS = 10

importing Jupyter notebook from InitialSetup.ipynb
Num GPUs Available:  1
Total images in validatation dataset:  12630
(32, 32, 3) (43,)
Total images in dataset:  39209


### Prepare dataset

In [2]:
BATCH_SIZE = 64

dataSolo = dataset
dataSolo = dataSolo.cache()
dataSolo = dataSolo.shuffle(buffer_size = dataset_length)
dataSolo = dataSolo.batch(batch_size = BATCH_SIZE)
dataSolo = dataSolo.prefetch(buffer_size = AUTOTUNE)

train_size = int(0.8* dataset_length)
val_size = int(0.2* dataset_length)

train_dataset = dataSolo.take(train_size)
val_dataset = dataSolo.skip(train_size)

file_path_prefix = './Networks/ensembles/V0/V0'    

### Setup models and save folder

In [ ]:
models_V0, histories_V0 = train_models(dataSolo, val_dataset, file_path_prefix, NUM_MODELS)

Epoch 1/20
490/490 [==============================] - 24s 30ms/step - loss: 3.5692 - accuracy: 0.2582 - val_loss: 1.4532 - val_accuracy: 0.5747

Epoch 00001: val_accuracy improved from -inf to 0.57470, saving model to ./Networks/ensembles/V0_00\cp.ckpt
Epoch 2/20
490/490 [==============================] - 13s 27ms/step - loss: 0.9632 - accuracy: 0.7207 - val_loss: 0.1901 - val_accuracy: 0.9526

Epoch 00002: val_accuracy improved from 0.57470 to 0.95262, saving model to ./Networks/ensembles/V0_00\cp.ckpt
Epoch 3/20
490/490 [==============================] - 14s 29ms/step - loss: 0.4896 - accuracy: 0.8471 - val_loss: 0.0980 - val_accuracy: 0.9771

Epoch 00003: val_accuracy improved from 0.95262 to 0.97713, saving model to ./Networks/ensembles/V0_00\cp.ckpt
Epoch 4/20
490/490 [==============================] - 14s 30ms/step - loss: 0.2978 - accuracy: 0.9106 - val_loss: 0.0574 - val_accuracy: 0.9864

Epoch 00004: val_accuracy improved from 0.97713 to 0.98641, saving model to ./Networks/ens

### Load Weights and evaluate models

In [4]:
models_V0 = load_weights_read(file_path_prefix, NUM_MODELS)
evaluate_models(models_V0, NUM_MODELS)

198/198 - 3s - loss: 0.0806 - accuracy: 0.9779
198/198 - 3s - loss: 0.1100 - accuracy: 0.9774
198/198 - 3s - loss: 0.1116 - accuracy: 0.9732
198/198 - 3s - loss: 0.0815 - accuracy: 0.9777
198/198 - 3s - loss: 0.0893 - accuracy: 0.9774
198/198 - 3s - loss: 0.0925 - accuracy: 0.9747
198/198 - 3s - loss: 0.0870 - accuracy: 0.9792
198/198 - 3s - loss: 0.0896 - accuracy: 0.9762
198/198 - 3s - loss: 0.0810 - accuracy: 0.9800
198/198 - 3s - loss: 0.0839 - accuracy: 0.9772
average accuracy: 97.708


In [7]:
labels_V0, logits_V0, preds_V0 = get_labels_logits_and_preds(models_V0, NUM_MODELS)
class_preds_V0 = get_class_preds(preds_V0, NUM_MODELS)
class_logits_V0 = get_class_from_sum_of_logits(logits_V0, NUM_MODELS)

### Build list of inputs and labels for test class

In [8]:
test_logits_preds = []

for i in range(test_dataset_length):
    
    aux = []   
    for m in range(NUM_MODELS):        
        aux.extend(logits_V0[m][i])
        
    test_logits_preds.append(aux)

### Compute logit predictions and training labels

In [9]:
dataSolo_len = len(dataSolo.take(-1))
j=0

logits_train = [[] for _ in range(NUM_MODELS)]
labels_aux = []
for images, labs in dataSolo.take(-1):

    #Printing progress
    print('\r' + f'{j}/{dataSolo_len}',end="")
    j+=1
    
    labels_aux.extend(labs.numpy())
    for i in range(NUM_MODELS):
        
        logits_train[i].extend(models_V0[i][1].predict(images))
        
labels_train = [np.argmax(i) for i in labels_aux]  

612/613

### Build list of train inputs

In [10]:
train_logits_preds = []

for i in range(dataset_length):
    
    aux = []
    
    for m in range(NUM_MODELS):
        
        aux.extend(logits_train[m][i])
        
    train_logits_preds.append(aux)

In [11]:
print(len(labels_train), len(train_logits_preds))

39209 39209


In [17]:

stack_model  = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(len(train_logits_preds[0]),)),

  tf.keras.layers.Dense(256),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(64),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),

  tf.keras.layers.Dense(43, activation='softmax')
])

In [18]:
stack_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

file_pathV7 = './Networks/ensembles/V0/stacked_V0/StackedDatasetOnly.ckpt'

callbacksV7 = prepare_callbacks(file_pathV7)

In [14]:
print(len(labels_train), len(train_logits_preds), len(logits_train[0]))

39209 39209 39209


In [19]:
stack_model.fit(np.asarray(train_logits_preds),np.asarray(labels_train),epochs=20, batch_size=BATCH_SIZE,
                        validation_data = (np.asarray(test_logits_preds), np.asarray(labels_V0)), callbacks = callbacksV7)

Epoch 1/20
613/613 [==============================] - 5s 7ms/step - loss: 3.0586 - accuracy: 0.2682 - val_loss: 1.2410 - val_accuracy: 0.8697

Epoch 00001: val_accuracy improved from -inf to 0.86968, saving model to ./Networks/ensembles/V0/stacked_V0\StackedDatasetOnly.ckpt
Epoch 2/20
613/613 [==============================] - 5s 7ms/step - loss: 1.4967 - accuracy: 0.7243 - val_loss: 0.7317 - val_accuracy: 0.9044

Epoch 00002: val_accuracy improved from 0.86968 to 0.90443, saving model to ./Networks/ensembles/V0/stacked_V0\StackedDatasetOnly.ckpt
Epoch 3/20
613/613 [==============================] - 5s 7ms/step - loss: 1.0023 - accuracy: 0.8179 - val_loss: 0.4649 - val_accuracy: 0.9327

Epoch 00003: val_accuracy improved from 0.90443 to 0.93270, saving model to ./Networks/ensembles/V0/stacked_V0\StackedDatasetOnly.ckpt
Epoch 4/20
613/613 [==============================] - 4s 7ms/step - loss: 0.7296 - accuracy: 0.8651 - val_loss: 0.3099 - val_accuracy: 0.9542

Epoch 00004: val_accuracy 

In [24]:
stack_model.load_weights(file_pathV7)

In [28]:
pred = stack_model.predict(np.asarray(test_logits_preds))

correct = 0

for i in range(test_dataset_length):
    if np.argmax(pred[i]) == labels_V0[i] :
        correct += 1
        
print(correct, correct/test_dataset_length)

12374 0.9797307996832938
